# Kaggle GAN Getting Started - Submission Notebook

This notebook generates Monet-style images for the competition.

## Instructions:
1. Upload your trained model to Kaggle Dataset
2. Modify the MODEL_PATH variable below
3. Run all cells
4. Download the generated images.zip


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import re
from tqdm import tqdm
import PIL
import shutil

# Configuration
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")


## Configuration

**IMPORTANT**: Change MODEL_PATH to your trained model path!


In [ ]:
# ===== CHANGE THIS TO YOUR MODEL PATH =====
MODEL_PATH = "../input/your-model-path/checkpoints/model_m_gen"
# =========================================

# Image parameters
HEIGHT, WIDTH, CHANNELS = 256, 256, 3
AUTO = tf.data.experimental.AUTOTUNE

print(f"Model path: {MODEL_PATH}")


## Helper Functions


In [ ]:
def count_data_items(filenames):
    """Count number of data items in TFRecord files"""
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)


def decode_image(image):
    """Decode JPEG image and normalize to [-1, 1]"""
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    return image


def read_tfrecord(example):
    """Read TFRecord example"""
    tfrecord_format = {
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image


def load_dataset(filenames):
    """Load dataset from TFRecord files"""
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    return dataset


## Load Photo Dataset


In [ ]:
# Find photo TFRecord files
GCS_PATH = "/kaggle/input/gan-getting-started"
PHOTO_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/photo_tfrec/*.tfrec'))

# Count and load
n_photo_samples = count_data_items(PHOTO_FILENAMES)
photo_ds = load_dataset(PHOTO_FILENAMES)

print(f"Found {len(PHOTO_FILENAMES)} photo TFRecord files")
print(f"Total photo samples: {n_photo_samples}")


## Load Trained Model


In [ ]:
# Load the trained Monet generator
print(f"Loading model from: {MODEL_PATH}")
try:
    loaded_model = tf.keras.models.load_model(MODEL_PATH)
    print("✓ Model loaded successfully!")
    print(f"Model input shape: {loaded_model.input_shape}")
    print(f"Model output shape: {loaded_model.output_shape}")
except Exception as e:
    print(f"✗ Error loading model: {e}")
    print("Please check your MODEL_PATH variable")


## Generate Monet-Style Images


In [ ]:
def predict_and_save(path, input_ds, generator_model):
    """Generate images and save to disk"""
    i = 1
    total = sum(1 for _ in input_ds)
    
    for img in tqdm(input_ds, total=total, desc="Generating images"):
        # Generate prediction
        prediction = generator_model(img, training=False)[0].numpy()
        
        # Rescale from [-1, 1] to [0, 255]
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
        
        # Save as JPEG
        im = PIL.Image.fromarray(prediction)
        im.save(os.path.join(path, "images", f"{i}.jpg"))
        i += 1
    
    return i - 1


In [ ]:
# Create output directory
output_dir = "./images"
os.makedirs(output_dir, exist_ok=True)

# Generate images
print("Starting image generation...")
num_generated = predict_and_save(
    path="./",
    input_ds=photo_ds.batch(1),
    generator_model=loaded_model
)

print(f"\n✓ Generated {num_generated} images")


## Create Submission ZIP


In [ ]:
# Create ZIP archive
images_path = os.path.join("./", 'images')
zip_path = shutil.make_archive(images_path, 'zip', images_path)

# Print summary
num_files = len([name for name in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, name))])
zip_size = os.path.getsize(zip_path) / (1024 * 1024)  # Size in MB

print(f"\n{'='*50}")
print(f"Submission Summary:")
print(f"  Generated images: {num_files}")
print(f"  ZIP file: {zip_path}")
print(f"  ZIP size: {zip_size:.2f} MB")
print(f"{'='*50}")

# Validation
if num_files < 7000:
    print("⚠ WARNING: Less than 7000 images generated!")
elif num_files > 10000:
    print("⚠ WARNING: More than 10000 images generated!")
else:
    print("✓ Image count is within acceptable range (7000-10000)")


## Next Steps

1. Download the `images.zip` file
2. Upload it to the competition submission page
3. Wait for evaluation results
